# 253.1.[실습] CNN활용_폴더에서 데이터수집_알츠하이머_MRI 분류

In [ ]:
# 출처: 모두의 딥러닝, 개정2판, 제20장

### 실습: 치매 환자의 뇌인지 일반인의 뇌인지 예측하기

## 1. 전이학습 없는 CNN 구현(ImageDataGenerator 활용- 데이터 증강 없음)

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

import numpy as np
import matplotlib.pyplot as plt

#학습셋의 변형을 설정하는 부분입니다. 


# 테스트셋은 이미지 부풀리기 과정을 진행하지 않습니다.

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [ ]:
# CNN 모델 구현


In [ ]:
# conda install SciPy
#!pip install SciPy

In [ ]:
#모델 최적화


# 조기 중단을 설정


#모델 학습


In [ ]:
# 검증셋과 학습셋의 오차 저장

# 그래프로 시각화
